In [ ]:
# pip uninstall -y trulens_eval
# pip install git+https://github.com/truera/trulens@piotrm/azure_bugfixes#subdirectory=trulens_eval

# trulens_eval notebook dev

%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

base = Path().cwd()
while not (base / "trulens_eval").exists():
    base = base.parent

print(base)

# If running from github repo, can use this:
sys.path.append(str(base))

# Uncomment for more debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

from trulens_eval.keys import check_keys

check_keys(
    "OPENAI_API_KEY",
    "HUGGINGFACE_API_KEY"
)

from trulens_eval import Tru
tru = Tru()
tru.reset_database()

tru.run_dashboard(_dev=base, force=True)

In [ ]:
%%writefile config.yaml
# Adapted from nemoguardrails/examples/bots/abc/config.yml
instructions:
  - type: general
    content: |
      Below is a conversation between a user and a bot called the trulens Bot.
      The bot is designed to answer questions about the trulens python library.
      The bot is knowledgeable about the library and python.
      If the bot does not know the answer to a question, it truthfully says it does not know.

sample_conversation: |
  user "Hi there. Can you help me with some questions I have about trulens?"
    express greeting and ask for assistance
  bot express greeting and confirm and offer assistance
    "Hi there! I'm here to help answer any questions you may have about the trulens. What would you like to know?"
  user "What integrations does trulens support?"
    ask question about integrations
  bot respond to question about integrations
    "The trulens library provides integration with langchain, llama_index, and nemo guardrails apps.Please refer to the documentation for more information."

models:
  - type: main
    engine: openai
    model: gpt-3.5-turbo-instruct

rails:
#  input:
#    flows:
#      - self check input

#  output:
#    flows:
#      - self check output

  dialog:
    single_call:
      enabled: False

In [ ]:
from langchain_core.language_models.base import BaseLanguageModel

from nemoguardrails import LLMRails, RailsConfig
from nemoguardrails.actions.llm.generation import LLMGenerationActions

config = RailsConfig.from_path(".")
rails = LLMRails(config)

In [ ]:
rails.kb

In [ ]:
import logging
from typing import Dict, List, Optional

from pprint import PrettyPrinter
pp = PrettyPrinter()
log = logging.getLogger(__name__)

from nemoguardrails.actions.actions import ActionResult, action

from trulens_eval import OpenAI, Huggingface, Feedback
hugs = Huggingface()

from trulens_eval.feedback.provider import OpenAI
import numpy as np

# Initialize provider class
openai = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(rails)

from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=OpenAI())
# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance)
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(openai.qs_relevance)
    .aggregate(np.mean)
)

feedbacks = dict()
feedbacks['language_match'] = Feedback(hugs.language_match)
feedbacks['answer_relevance'] = f_qa_relevance
feedbacks['groundedness'] = f_groundedness
feedbacks['context_relevance'] = f_context_relevance

In [ ]:
@action(name="feedback")
async def feedback(
    events: Optional[List[Dict]] = None, 
    context: Optional[dict] = None,
    llm: Optional[BaseLanguageModel] = None,
    config: Optional[RailsConfig] = None,
    function: Optional[str] = None,
    selectors: Optional[Dict[str, Lens]] = None
):
    """Run the specified feedback function from trulens_eval.

    Args:
        context (Optional[dict]): The context for the execution of the action.
        Defaults to None.

    Returns:
        ActionResult: An action result containing the result of the feedback.

    Note:
        ...

    Example:
        ```python
        result = await run_feedback(context=my_context)
        print(result.result)  # Score of the feedback function (between 0.0 and 1.0)
        ```
    """

    feedback_function = feedbacks.get(function)
    if feedback_function is None:
        raise ValueError(f"Invalid feedback function: {function}")

    if selectors is None:
        raise ValueError(f"Need selectors for feedback function: {feedback_function}")
    
    selectors = {argname: Lens.of_string(arglens) for argname, arglens in selectors.items()}

    feedback_function = feedback_function.on(**selectors)

    source_data = dict(
        action=dict(events=events, context=context, llm=llm, config=config)
    )

    context_updates = {}
    
    try:
        result = feedback_function.run(source_data=source_data)
    except Exception as e:
        context_updates["result"] = 1.0

        return ActionResult(
            return_value=context_updates["result"],
            context_updates=context_updates,
        )

    if result.result is None:
        context_updates["result"] = 1.0
    else:
        context_updates["result"] = result.result

    print(f"{feedback_function.name} = {result.result}")

    return ActionResult(
        return_value=context_updates["result"],
        context_updates=context_updates,
    )

In [ ]:
rails.register_action(feedback)


In [ ]:
response = await rails.generate_async(messages=[{
    "role": "user",
    "content": "What does the company handbook say about pasta?"
}])
    
print(response)

In [ ]:
from langchain_core.language_models.base import BaseLanguageModel

from nemoguardrails import LLMRails, RailsConfig
from nemoguardrails.actions.llm.generation import LLMGenerationActions

config = RailsConfig.from_path(".")
rails = LLMRails(config)

In [ ]:
rails.kb

In [ ]:
from trulens_eval.feedback.feedback import RAG_triad, Feedback
from trulens_eval.feedback.provider import OpenAI
from trulens_eval.schema import Select
from trulens_eval.tru_rails import RailsActionSelect, TruRails

# Initialize provider class
openai = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App

context = App.select_context(rails)
question = Select.RecordInput
answer = Select.RecordOutput

triad = RAG_triad(
    provider=openai, question=question, answer=answer, context=context
)

In [ ]:
tru_rails = TruRails(rails, feedbacks=triad)

In [ ]:
tru_rails.print_instrumented()

In [ ]:
with tru_rails as recorder:
    response = await rails.generate_async(messages=[{
    "role": "user",
    "content": "Which frameworks does trulens support?"
}])
    
print(response)

In [ ]:
rec = recorder.get()